In [33]:
import pandas as pd
import scipy.stats as ss
import numpy as np

df = pd.read_stata('hsbdemo.dta')
df = df.ix[:,1:]
df = df.ix[:,:-1]
df[199:]

,female,ses,schtyp,prog,read,write,math,science,socst,honors,awards
199,male,middle,public,academic,73.0,62.0,73.0,69.0,66.0,enrolled,3.0


In [34]:
y_data = pd.get_dummies(df.ix[:,"prog"]).as_matrix()
x_data_df = df.drop(["prog"],axis=1)

In [37]:
import scipy.stats as ss

x_data = pd.get_dummies(x_data_df).as_matrix()
x_data[:,1:] = np.array(ss.zscore(x_data[:,1:]))

training_data_rate = 0.7
random_index = np.random.choice(len(x_data), len(x_data))
training_data_split_index = int(len(random_index) * training_data_rate) 

training_x_data = x_data[:training_data_split_index]
test_x_data = x_data[training_data_split_index:]

training_y_data = y_data[:training_data_split_index]
test_y_data = y_data[training_data_split_index:]

In [39]:
import tensorflow as tf

learning_rate = 0.001

# Network Parameters
n_hidden_1 = 16 # 1st layer number of features
n_hidden_2 = 1 # 2nd layer number of features
n_hidden_3 = 1 # 3th layer number of features
n_hidden_4 = 256 # 4th layer number of features
n_hidden_5 = 16 # 5th layer number of features

n_input = len(x_data[0])
n_classes = len(y_data[0])

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
dropout_rate = tf.placeholder("float")

def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    
    
    _layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    layer_1 = tf.nn.dropout(_layer_1, dropout_rate)
    _layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    layer_2 = tf.nn.dropout(_layer_2, dropout_rate)
    _layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['h3']), biases['b3']))
    layer_3 = tf.nn.dropout(_layer_3, dropout_rate)
    _layer_4 = tf.nn.relu(tf.add(tf.matmul(layer_3, weights['h4']), biases['b4']))
    layer_4 = tf.nn.dropout(_layer_4, dropout_rate)
    _layer_5 = tf.nn.relu(tf.add(tf.matmul(layer_4, weights['h5']), biases['b5']))
    layer_5 = tf.nn.dropout(_layer_5, dropout_rate)
    
    out_layer = tf.matmul(layer_5, weights['out']) + biases['out']
    return out_layer

weights = {
#     'h1': tf.get_variable("W1", shape = [n_input, n_hidden_1], initializer=tf.contrib.layers.xavier_initializer()),
#     'h2': tf.get_variable("W2", shape = [n_hidden_1, n_hidden_2], initializer=tf.contrib.layers.xavier_initializer()),
#     'h3': tf.get_variable("W3", shape = [n_hidden_2, n_hidden_3], initializer=tf.contrib.layers.xavier_initializer()),
#     'h4': tf.get_variable("W4", shape = [n_hidden_3, n_hidden_4], initializer=tf.contrib.layers.xavier_initializer()),
#     'h5': tf.get_variable("W5", shape = [n_hidden_4, n_hidden_5], initializer=tf.contrib.layers.xavier_initializer()),
#     'out': tf.get_variable("out", shape = [n_hidden_5, n_classes], initializer=tf.contrib.layers.xavier_initializer())


    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4])),
    'h5': tf.Variable(tf.random_normal([n_hidden_4, n_hidden_5])),
    'out': tf.Variable(tf.random_normal([n_hidden_5, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'b5': tf.Variable(tf.random_normal([n_hidden_5])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for i in range(1000):
        _, c = sess.run([optimizer, cost], feed_dict={x: training_x_data, y: training_y_data, dropout_rate: 0.9})
    print ("Optimization Finished!")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy:", accuracy.eval({x: test_x_data, y: test_y_data, dropout_rate: 1}))

Optimization Finished!
Accuracy: 0.733333
